In [1]:
import pandas as pd
import numpy as np
import matplotlib
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression

In [2]:
df = pd.read_csv('./data/train.csv')


In [3]:
# split cabin into num let
cabin_loc_let = []
cabin_loc_num = []

for i in df['Cabin']:
    x = str(i)
    cabin_loc_let.append(x[0])
    if x[1] == 'a':
        cabin_loc_num.append(0)
    else:
        cabin_loc_num.append(x[1:])
        
df['cabin_loc_let'] = cabin_loc_let
df['cabin_loc_num'] = cabin_loc_num

In [4]:
# label encoder
cll_le = preprocessing.LabelEncoder() 
cll_enc = cll_le.fit_transform(df['cabin_loc_let'])
sex_le = preprocessing.LabelEncoder() 
sex_enc = sex_le.fit_transform(df['Sex'])
pclass_le = preprocessing.LabelEncoder() 
pclass_enc = pclass_le.fit_transform(df['Pclass'])
embarked_le = preprocessing.LabelEncoder() 
embarked_enc = embarked_le.fit_transform(df['Embarked'])

In [5]:
# one hot Pclass, Embarked, Cabin, Ticket, Sex
enc = OneHotEncoder(sparse = True)

cll_enc = cll_enc.reshape(-1,1)
sex_enc = sex_enc.reshape(-1,1)
pclass_enc = pclass_enc.reshape(-1,1)
embarked_enc = embarked_enc.reshape(-1,1)


cll_onehot = enc.fit_transform(cll_enc).toarray()
cll_cat = enc.categories_
sex_onehot = enc.fit_transform(sex_enc).toarray()
sex_cat = enc.categories_
pclass_onehot = enc.fit_transform(pclass_enc).toarray()
pclass_cat = enc.categories_
embarked_onehot = enc.fit_transform(embarked_enc).toarray()
embarked_cat = enc.categories_


In [6]:
# get column names
cll_cat = np.asarray(cll_cat)
sex_cat = np.asarray(sex_cat)
pclass_cat = np.asarray(pclass_cat)
embarked_cat = np.asarray(embarked_cat)

cll_cat = cll_cat.reshape(-1,1)
sex_cat = sex_cat.reshape(-1,1)
pclass_cat = pclass_cat.reshape(-1,1)
embarked_cat = embarked_cat.reshape(-1,1)

cll_inv = cll_le.inverse_transform(cll_cat)
sex_inv = sex_le.inverse_transform(sex_cat)
pclass_inv = pclass_le.inverse_transform(pclass_cat)
embarked_inv = embarked_le.inverse_transform(embarked_cat)


c:\users\jimmy\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [7]:
cll_df = pd.DataFrame(data = cll_onehot, columns = cll_inv)
sex_df = pd.DataFrame(data = sex_onehot, columns = sex_inv)
pclass_df = pd.DataFrame(data = pclass_onehot, columns = pclass_inv)
embarked_df = pd.DataFrame(data = embarked_onehot, columns = embarked_inv)

df2 = pd.concat([df, cll_df, sex_df, pclass_df, embarked_df], axis = 1)


In [8]:
df2 = df2.drop(['Ticket', 'Sex', 'Cabin', 'Embarked', 'Pclass', 'Name', 'cabin_loc_let'], axis = 1)

In [9]:
# replace nan
# df2.columns[df2.isna().any()].tolist()
mean_age = df2['Age'].mean()
mean_fare = df2['Fare'].mean()
df2['Age'] = df2['Age'].fillna(value = mean_age)
df2['Fare'] = df2['Fare'].fillna(value = mean_fare)


df2.columns[df2.isna().any()].tolist()



[]

In [10]:
df2.mean()

PassengerId    49999.500000
Survived           0.427740
Age               38.355472
SibSp              0.397690
Parch              0.454560
Fare              43.929330
A                  0.063070
B                  0.074390
C                  0.118250
D                  0.036370
E                  0.017490
F                  0.006630
G                  0.004820
T                  0.000320
n                  0.678660
female             0.438860
male               0.561140
1                  0.303150
2                  0.286790
3                  0.410060
C                  0.221870
Q                  0.054240
S                  0.721390
NaN                0.002500
dtype: float64

In [11]:
corr = df2.corr()
corr

,PassengerId,Survived,Age,SibSp,Parch,Fare,A,B,C,D,...,n,female,male,1,2,3,C,Q,S,NaN
PassengerId,1.000000,0.001956,-0.002611,0.001000,0.000411,-0.001035,0.001417,0.004412,-0.005680,0.005904,...,-0.002142,-0.000493,0.000493,0.001125,-0.000632,-0.000470,0.004279,-0.003042,-0.002929,0.004489
Survived,0.001956,1.000000,0.102186,-0.007587,0.023352,0.187407,-0.080300,0.116006,0.210568,0.119552,...,-0.251554,0.507260,-0.507260,0.202688,0.124682,-0.304056,0.349209,0.081917,-0.365716,0.006098
Age,-0.002611,0.102186,1.000000,-0.110755,-0.123663,0.107210,0.063726,0.118143,0.115095,0.082320,...,-0.229933,0.080341,-0.080341,0.227625,0.009389,-0.221344,0.100214,0.150474,-0.169231,0.002978
SibSp,0.001000,-0.007587,-0.110755,1.000000,0.213126,0.059336,0.117740,0.066607,-0.030399,-0.021885,...,-0.066946,0.030627,-0.030627,0.052492,-0.011416,-0.038555,-0.019017,-0.030716,0.033464,-0.002884
Parch,0.000411,0.023352,-0.123663,0.213126,1.000000,0.061414,0.092058,0.129943,-0.055094,-0.023402,...,-0.076096,0.087666,-0.087666,0.043466,-0.010830,-0.030660,0.014203,-0.007879,-0.009520,0.003027
Fare,-0.001035,0.187407,0.107210,0.059336,0.061414,1.000000,0.124524,0.202063,0.354341,0.091120,...,-0.464100,0.143458,-0.143458,0.461509,-0.165472,-0.279116,0.211763,0.101287,-0.247947,0.004593
A,0.001417,-0.080300,0.063726,0.117740,0.092058,0.124524,1.000000,-0.073553,-0.095014,-0.050405,...,-0.377052,-0.055449,0.055449,0.303596,-0.107494,-0.184862,0.005907,-0.036342,0.012864,0.000192
B,0.004412,0.116006,0.118143,0.066607,0.129943,0.202063,-0.073553,1.000000,-0.103818,-0.055076,...,-0.411990,0.094789,-0.094789,0.361911,-0.146907,-0.203114,0.038935,0.351747,-0.214164,0.003360
C,-0.005680,0.210568,0.115095,-0.030399,-0.055094,0.354341,-0.095014,-0.103818,1.000000,-0.071145,...,-0.532196,0.127838,-0.127838,0.479018,-0.188191,-0.274588,0.280796,0.011843,-0.266604,0.003372
D,0.005904,0.119552,0.082320,-0.021885,-0.023402,0.091120,-0.050405,-0.055076,-0.071145,1.000000,...,-0.282332,0.081254,-0.081254,0.181120,-0.046187,-0.126783,0.147463,0.046633,-0.160338,0.000971


In [16]:
# fare, age, cabin letters, male, female, 1, 2, 3, C, Q, s

df2['female'].sum()

43886.0

In [15]:
df2['male'].sum()


56114.0